In the first part of the project, you should work on retrieving, structuring, and cleaning data. Document your work in a jupyter notebook and in the final project report.

You will be using a subset of the FakeNewsCorpus dataset in your project, which is available from Absalon. You can also find more information about the full datasetLinks to an external site. and find information about how the data is collected, the available fields, etc.

Task 1: Your first task is to retrieve a sample of the FakeNewsCorpus from https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csvLinks to an external site. and structure, process, clean it. You should follow the methodology you developed in Exercise 1. When you have finished cleaning, you can start to process the text. NLTK (https://www.nltk.org/ Links to an external site.) has built-in support for many common operations. Try the following:

Tokenize the text.
Remove stopwords and compute the size of the vocabulary. Compute the reduction rate of the vocabulary size after removing stopwords.
Remove word variations with stemming and compute the size of the vocabulary. Compute the reduction rate of the vocabulary size after stemming.
Describe which procedures (and which libraries) you used and why they are appropriate.

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import Counter
from cleantext import clean
import re as regular_expression
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bruger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Task 1, part 1: Her åbner vi datafilen, som vi skal strukturere, behandle og cleane. 

In [2]:
df = pd.read_csv('news_sample.csv')
df

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary
0,0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,Sometimes the power of Christmas will make you...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Church Congregation Brings Gift to Waitresses ...,Ruth Harris,NaN,[''],NaN,NaN,NaN
1,1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,Zurich Times,NaN,[''],NaN,NaN,NaN
2,2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,Never Hike Alone: A Friday the 13th Fan Film U...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Never Hike Alone - A Friday the 13th Fan Film ...,NaN,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN,NaN
3,3,768,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,"When a rare shark was caught, scientists were ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Elusive ‘Alien Of The Sea ‘ Caught By Scientis...,Alexander Smith,NaN,[''],NaN,NaN,NaN
4,4,791,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,Donald Trump has the unnerving ability to abil...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Trump’s Genius Poll Is Complete & The Results ...,Gloria Christie,NaN,[''],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,39259,beforeitsnews.com,fake,http://beforeitsnews.com/economy/2017/12/priso...,"Prison for Rahm, God’s Work And Many Others\n\...",2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Prison for Rahm, God’s Work And Many Others",NaN,NaN,[''],NaN,NaN,NaN
246,246,39468,beforeitsnews.com,fake,http://beforeitsnews.com/diy/2017/11/4-useful-...,4 Useful Items for Your Tiny Home\n\nHeadline:...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,4 Useful Items for Your Tiny Home,Dimitry K,NaN,[''],NaN,NaN,NaN
247,247,39477,www.newsmax.com,NaN,https://www.newsmax.com/politics/michael-hayde...,Former CIA Director Michael Hayden said Thursd...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Michael Hayden: We Should Be 'Frightened' by T...,Todd Beamon,NaN,"['michael hayden', 'sthole countries', 'daca',...",President Donald Trump's reported remarks abou...,"Homeland Security, Trump Administration, Immig...",NaN
248,248,39550,www.newsmax.com,NaN,https://www.newsmax.com/newsfront/antonio-saba...,Antonio Sabato Jr. says Hollywood's liberal el...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Antonio Sabato Jr.: It's Oprah or Bust for Hol...,Bill Hoffmann,NaN,"['antonio sabato jr', 'oprah winfrey', 'presid...",Antonio Sabato Jr. says Hollywood's liberal el...,"Trump Administration, ISIS/Islamic State, News...",NaN


In [7]:
# Bruger denne funktion til at cleance vores datafil, Tal bliver lavet om til "<NUMBER>".
# Links bliver lavet om til "<URL>", mails bliver lavet om til "<EMAIL>"", telefonnumre bliver omdannet til "<PHONE>".
# Alle store bogstaver bliver omdannet til små bogstaver og alle mellemrum bliver fjernet.
# Alle tal/numre bliver omdannet til "<NUMBER>"" eller "0"
def apply_clean(text):
    return clean(text,
         fix_unicode=False,
         to_ascii=False,
         lower=True,
         no_line_breaks=True,
         no_urls=True,
         no_emails=True,
         no_phone_numbers=True,
         no_numbers=True,
         no_digits=True,
         replace_with_url="<URL>",
         replace_with_email="<EMAIL>",
         replace_with_phone_number="<PHONE>",
         replace_with_number="<NUMBER>",
         replace_with_digit="0",
         lang="en"
        )

df_cleaned = df.applymap(apply_clean)

print(df_cleaned)

    Unnamed: 0        id                domain        type    url  \
0     <number>  <number>               awm.com  unreliable  <url>   
1     <number>  <number>     beforeitsnews.com        fake  <url>   
2     <number>  <number>           cnnnext.com  unreliable  <url>   
3     <number>  <number>               awm.com  unreliable  <url>   
4     <number>  <number>  bipartisanreport.com   clickbait  <url>   
..         ...       ...                   ...         ...    ...   
245   <number>  <number>     beforeitsnews.com        fake  <url>   
246   <number>  <number>     beforeitsnews.com        fake  <url>   
247   <number>  <number>                 <url>         nan  <url>   
248   <number>  <number>                 <url>         nan  <url>   
249   <number>  <number>                 <url>         nan  <url>   

                                               content  \
0    sometimes the power of christmas will make you...   
1    awakening of <number> strands of dna – “reconn... 

Processing:

In [31]:
# Vi anvender rensningsfunktionen på 'content' kolonnen
df['cleaned_content'] = df['content'].apply(apply_clean)

# Tokenisering af den rensede 'content' kolonne
df['content_tokens'] = df['cleaned_content'].apply(word_tokenize)

# Fjernelse af stopord
stop_words = set(stopwords.words('english'))
df['content_tokens_no_stop'] = df['content_tokens'].apply(lambda tokens: [token for token in tokens if token not in stop_words])

# Anvendelse af stemming
stemmer = PorterStemmer()
df['content_tokens_stemmed'] = df['content_tokens_no_stop'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

# Beregning af størrelsen på ordforrådet og reduktionsrater
original_vocab_size = df['content_tokens'].explode().nunique()
vocab_size_no_stop = df['content_tokens_no_stop'].explode().nunique()
vocab_size_stemmed = df['content_tokens_stemmed'].explode().nunique()

stopwords_reduction_rate = calculate_reduction_rate(original_vocab_size, vocab_size_no_stop)
stemming_reduction_rate = calculate_reduction_rate(vocab_size_no_stop, vocab_size_stemmed)

# Viser de beregnede størrelser og reduktionsrater
print(f'Original Vocabulary Size: {original_vocab_size}')
print(f'Vocabulary Size after removing stopwords: {vocab_size_no_stop}')
print(f'Reduction rate after removing stopwords: {stopwords_reduction_rate:.2f}')
print(f'Vocabulary Size after stemming: {vocab_size_stemmed}')
print(f'Reduction rate after stemming: {stemming_reduction_rate:.2f}')

# Viser de første par rækker for at bekræfte ændringerne
df[['content_tokens', 'content_tokens_no_stop', 'content_tokens_stemmed']].head()


Original Vocabulary Size: 17913
Vocabulary Size after removing stopwords: 17767
Reduction rate after removing stopwords: 0.01
Vocabulary Size after stemming: 12603
Reduction rate after stemming: 0.29


,content_tokens,content_tokens_no_stop,content_tokens_stemmed
0,"[sometimes, the, power, of, christmas, will, m...","[sometimes, power, christmas, make, wild, wond...","[sometim, power, christma, make, wild, wonder,..."
1,"[awakening, of, 12, strands, of, dna, –, “, re...","[awakening, 12, strands, dna, –, “, reconnecti...","[awaken, 12, strand, dna, –, “, reconnect, ”, ..."
2,"[never, hike, alone, :, a, friday, the, 13th, ...","[never, hike, alone, :, friday, 13th, fan, fil...","[never, hike, alon, :, friday, 13th, fan, film..."
3,"[when, a, rare, shark, was, caught, ,, scienti...","[rare, shark, caught, ,, scientists, left, blu...","[rare, shark, caught, ,, scientist, left, blun..."
4,"[donald, trump, has, the, unnerving, ability, ...","[donald, trump, unnerving, ability, ability, c...","[donald, trump, unnerv, abil, abil, creat, rea..."
